In [1]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [2]:
# --- QUANTUM LOOM: 4-BODY SOLAR SYSTEM SCAN (WITH SATURN) ---
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
import numpy as np

# 2. DEFINE THE GIANTS
# Jupiter
M_JUPITER = 317.8
D_JUPITER = 5.2
PHASE_J   = np.log(M_JUPITER + 1) * np.pi

# Saturn (The New Player)
M_SATURN  = 95.2
D_SATURN  = 9.5
PHASE_S   = np.log(M_SATURN + 1) * np.pi

# The Scan (Inner Belt to Outer Belt)
scan_distances = np.linspace(1.8, 3.6, 30)

# 3. BUILD CIRCUITS
circuits = []
for dist in scan_distances:
    # 4 Qubits: Sun(0), Jupiter(1), Saturn(2), Asteroid(3)
    qc = QuantumCircuit(4)
    qc.x(0) # Sun
    
    # Initialize Superpositions
    qc.h(1) # Jupiter
    qc.h(2) # Saturn
    qc.h(3) # Asteroid
    
    # Apply Mass Phases
    qc.rz(PHASE_J, 1)
    qc.rz(PHASE_S, 2)
    qc.rz(np.log(0.0001+1)*np.pi, 3) # Tiny Asteroid
    
    # --- VACUUM TENSION (SUN <-> BODY) ---
    qc.rzz(D_JUPITER * 0.1 * np.pi, 0, 1)
    qc.rzz(D_SATURN  * 0.1 * np.pi, 0, 2)
    qc.rzz(dist      * 0.1 * np.pi, 0, 3)
    
    # --- JUPITER DISRUPTION (1 <-> 3) ---
    # Strong Interaction
    str_j = (M_JUPITER) / (abs(D_JUPITER - dist)**2)
    coup_j = np.clip(str_j * 0.005 * np.pi, 0, np.pi)
    qc.rzz(coup_j, 1, 3)
    
    # --- SATURN DISRUPTION (2 <-> 3) ---
    # Weaker, but significant harmonic interaction
    str_s = (M_SATURN) / (abs(D_SATURN - dist)**2)
    coup_s = np.clip(str_s * 0.005 * np.pi, 0, np.pi)
    qc.rzz(coup_s, 2, 3)
    
    qc.h(3)
    qc.measure_all()
    circuits.append(qc)

# 4. RUN
print(f"Scanning {len(scan_distances)} slots with 4-Body Physics...")
isa_circuits = transpile(circuits, backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=2048)
res = job.result()

# 5. ANALYZE & PLOT
print("\n=== THE SATURN-JUPITER ALLIANCE MAP ===")
print("Dist (AU) | Stability | Graph")

stabilities = []
for i, dist in enumerate(scan_distances):
    counts = res[i].data.meas.get_counts()
    total = sum(counts.values())
    
    # Check Qubit 3 (Asteroid) - Leftmost bit in string
    zeros = 0
    for k, v in counts.items():
        if k[0] == '0': # Q3 is MSB
            zeros += v
            
    stab = 2 * abs((zeros/total) - 0.5)
    stabilities.append(stab)
    
    bar = "#" * int(stab * 20)
    print(f"{dist:.2f}      | {stab*100:.0f}%       | {bar}")

Scanning 30 slots with 4-Body Physics...

=== THE SATURN-JUPITER ALLIANCE MAP ===
Dist (AU) | Stability | Graph
1.80      | 76%       | ###############
1.86      | 69%       | #############
1.92      | 70%       | ##############
1.99      | 71%       | ##############
2.05      | 66%       | #############
2.11      | 68%       | #############
2.17      | 67%       | #############
2.23      | 60%       | ############
2.30      | 63%       | ############
2.36      | 57%       | ###########
2.42      | 57%       | ###########
2.48      | 56%       | ###########
2.54      | 51%       | ##########
2.61      | 48%       | #########
2.67      | 46%       | #########
2.73      | 48%       | #########
2.79      | 42%       | ########
2.86      | 40%       | ########
2.92      | 31%       | ######
2.98      | 29%       | #####
3.04      | 26%       | #####
3.10      | 24%       | ####
3.17      | 20%       | ###
3.23      | 12%       | ##
3.29      | 12%       | ##
3.35      | 2%       | 
3.41   